In [3]:
import numpy as np
import pandas as pd
import json

In [5]:
data = pd.read_csv("table_scans.csv", sep='|')
data

,QUERY_HASH,COLUMN_TYPE,TABLE_NAME,COLUMN_NAME,INPUT_ROWS,OUTPUT_ROWS,RUNTIME_NS,DESCRIPTION
0,4de6029635fed157,REFERENCE,customer,c_phone,680672,190833,150963713,TableScan Impl: ExpressionEvaluator (SUBSTR(c_...
1,4de6029635fed157,REFERENCE,customer,c_acctbal,1500000,680672,282989251,TableScan Impl: ColumnVsValue c_acctbal > SUBQ...
2,eddfe86e9c0c6462,REFERENCE,customer,c_phone,680792,190810,149327106,TableScan Impl: ExpressionEvaluator (SUBSTR(c_...
3,eddfe86e9c0c6462,REFERENCE,customer,c_acctbal,1500000,680792,282469772,TableScan Impl: ColumnVsValue c_acctbal > SUBQ...
4,df8208d52a5dee04,REFERENCE,customer,c_phone,681030,190684,148784923,TableScan Impl: ExpressionEvaluator (SUBSTR(c_...
...,...,...,...,...,...,...,...,...
7123,a515c8ac5ddb71e8,DATA,lineitem,l_shipdate,800000,698269,1153036,TableScan Impl: ColumnBetween l_shipdate BETWE...
7124,51102c16cd07dda9,DATA,lineitem,l_shipdate,59500000,59429315,133587040,TableScan Impl: ColumnVsValue l_shipdate <= '1...
7125,3f7329654f9c92e,DATA,lineitem,l_shipdate,800000,748161,1278004,TableScan Impl: ColumnBetween l_shipdate BETWE...
7126,40e12c9988ffa58d,REFERENCE,lineitem,l_shipdate,2400000,2269014,14054269,TableScan Impl: ColumnBetween l_shipdate BETWE...


In [2]:
import numpy as np
import pandas as pd
import json

DEFAULT_CHUNK_SIZE = 50000
primary_key_columns = {
            'supplier': 's_suppkey',
            'part': 'p_partkey',
            'partsupp': 'ps_partkey',
            'customer': 'c_custkey',
            'orders': 'o_orderkey',
            'lineitem': 'l_orderkey',
        }

def score(table_scans_for_column):
    runtimes = table_scans_for_column['RUNTIME_NS']
    output_rows = table_scans_for_column['OUTPUT_ROWS']
    output_rows = output_rows + (output_rows % DEFAULT_CHUNK_SIZE) # round it to the next multiple
    scores = runtimes - (runtimes * (output_rows / table_scans_for_column['INPUT_ROWS']))
    return scores.sum()

def cluster_column_for_table(data, table_name):
    table_scans = data[data['TABLE_NAME'] == table_name]
    columns = np.unique(table_scans['COLUMN_NAME'])
    print("CALCULATING SCORES FOR TABLE ", table_name)
    scores = [score(table_scans[table_scans['COLUMN_NAME'] == column]) for column in columns]
    s = np.array(scores)
    s /= s.mean() # can't make sense out of such big numbers..
    scores = s.tolist()

    scores = sorted([(column, sc) for sc, column in zip(scores, columns)], key=lambda s: s[1], reverse=True)
    print(scores)
    
    cluster_columns = []
    
    if len(scores) == 0:
        return None
    
    previous_score = scores[0][1]
    for i in range(len(scores)):
        sc = scores[i]
        if (sc[1] / previous_score) / max(i, 1) < 0.25:
            break
        else:
            cluster_columns.append(sc)
            previous_score = sc[1]
        
    
    print(cluster_columns)
    return [c[0] for c in cluster_columns]
    
def determine_clusters(data):
    
    data = data[data['COLUMN_TYPE'] == 'DATA']
    
    tables = np.unique(data['TABLE_NAME'])
    print(tables)
    clusters = {}
    
    for table_name in tables:
        column_names = cluster_column_for_table(data, table_name)
        table_clusters = []
        avg_table_size = np.array([data[data['COLUMN_NAME'] == column_name]['INPUT_ROWS'].mean() for column_name in column_names]).mean()
        
        if avg_table_size < DEFAULT_CHUNK_SIZE:
            continue
        
        combined_cluster_count = avg_table_size / DEFAULT_CHUNK_SIZE

        
        clusters[table_name] = {
            'cluster_columns': column_names if len(column_names) > 0 else [primary_key_columns[table_name]],
            'sort_column': primary_key_columns[table_name]
        }
        
        
    with open("clustering.json", "w") as f:
        f.write(json.dumps(clusters))


data = pd.read_csv("table_scans.csv", sep='|')
determine_clusters(data)

['customer' 'lineitem' 'nation' 'orders' 'part' 'region']
CALCULATING SCORES FOR TABLE  customer
[('c_mktsegment', 1.0)]
[('c_mktsegment', 1.0)]
CALCULATING SCORES FOR TABLE  lineitem
[('l_shipmode', 2.103182794851919), ('l_discount', 0.8822871105809325), ('l_shipdate', 0.014530094567148594)]
[('l_shipmode', 2.103182794851919), ('l_discount', 0.8822871105809325)]
CALCULATING SCORES FOR TABLE  nation
[('n_name', 1.0)]
[('n_name', 1.0)]
CALCULATING SCORES FOR TABLE  orders
[('o_orderdate', 1.7195415129436449), ('o_comment', 0.8551179105786682), ('o_orderstatus', 0.4253405764776869)]
[('o_orderdate', 1.7195415129436449), ('o_comment', 0.8551179105786682)]
CALCULATING SCORES FOR TABLE  part
[('p_name', 4.437841975050041), ('p_container', 0.1920606094784841), ('p_size', 0.19175396100680972), ('p_type', 0.15608140989297317), ('p_brand', 0.022262044571690575)]
[('p_name', 4.437841975050041)]
CALCULATING SCORES FOR TABLE  region
[('r_name', 1.0)]
[('r_name', 1.0)]


In [17]:
for table in np.unique(data['TABLE_NAME']):
    print(table)
    z = data[data['COLUMN_TYPE'] == 'DATA']
    d = z[z['TABLE_NAME'] == table]
    c = np.unique(d['COLUMN_NAME'])
    for u in c:
        print(d[d['COLUMN_NAME'] == u]['INPUT_ROWS'])

customer
2823    1500000
3050    1500000
3122    1500000
3125    1500000
3128    1500000
         ...   
6447    1500000
6604    1500000
6665    1500000
6763    1500000
7019    1500000
Name: INPUT_ROWS, Length: 76, dtype: int64
lineitem
2667    9200000
2760    9200000
2812    9200000
2826    9200000
2829    9300000
         ...   
6120    9200000
6281    9200000
6497    9200000
6955    9200000
7080    9200000
Name: INPUT_ROWS, Length: 61, dtype: int64
2004     2400000
2005     2400000
2013     2400000
2014     2400000
2020     2300000
          ...   
7088      900000
7123      800000
7124    59500000
7125      800000
7127     2400000
Name: INPUT_ROWS, Length: 200, dtype: int64
2603     9900000
2604     9900000
2838     9900000
2839     9900000
2881     9900000
          ...   
7055     9900000
7086     9900000
7087     9900000
7121    10000000
7122    10000000
Name: INPUT_ROWS, Length: 168, dtype: int64
nation
73      25
79      25
85      25
93      25
99      25
        ..
6566    2